In [1]:
import sys
import os
sys.path.insert(0,os.path.abspath("../lib/"))

import json

from util import JSONExtendedEncoder

from modulecontainers import Modules

%load_ext autoreload
%autoreload 2

%matplotlib inline
from matplotlib.pyplot import *

import pandas as pd
import numpy as np

import multiprocessing as mp

from collections import OrderedDict

from itertools import product
from collections import defaultdict

import itertools
import shutil

conf_folder = "conf/"

from cluster import generate_batchcode
import prism_cluster
import os

import pyperclip as pc

In [2]:
from modulescomparison import ModevalFunctional, ModevalKnownmodules, ModevalCoverage

In [6]:
methodname = "agglom"
settings_name = "paramexplo/" + methodname
settings = json.load(open("../conf/settings/{}.json".format(settings_name)))

In [4]:
# create pool of processors
if "pool" in locals().keys():
    pool.close()
pool = mp.Pool(mp.cpu_count()-1)

In [7]:
modeval_functional = ModevalFunctional(settings)

In [8]:
modeval_functional.run(pool)

Evaluating a total of 756 settings.
>>0.03416252136230469
>>0.07155871391296387
>>0.05527830123901367
>>0.07154011726379395
>>0.07773756980895996
>>0.1174156665802002
>>0.2171647548675537
>>0.026733875274658203>>0.05980110168457031>>0.14481353759765625>>0.08236908912658691>>0.1073760986328125>>0.12765979766845703>>0.21063446998596191






>>0.08549022674560547>>0.06447291374206543>>0.16400837898254395>>0.20337486267089844>>0.21326780319213867>>0.06445741653442383>>0.13553929328918457






>>0.07177138328552246>>0.09694194793701172>>0.06753730773925781>>0.1839292049407959>>0.31058454513549805>>0.058118343353271484>>0.10994911193847656






>>0.0387568473815918>>0.25238800048828125>>0.08973574638366699>>0.06960701942443848>>0.12548470497131348>>0.050786495208740234>>0.0874640941619873






>>0.03059101104736328>>0.46399497985839844>>0.0849301815032959>>0.08415627479553223>>0.08191251754760742>>0.08069539070129395>>0.08229708671569824






>>0.03463888168334961>>0.05623340606689453>>

In [15]:
modeval_functional.save(settings_name)

In [20]:
scores = {}
for methodname in ["dummy", "agglom"]:
    settings_name = "paramexplo/" + methodname
    settings = json.load(open("../conf/settings/{}.json".format(settings_name)))
    settings_dataset = pd.DataFrame([dict(settingid=settingid, **json.load(open("../" + setting["dataset_location"]))["params"]) for settingid, setting in enumerate(settings)])
    settings_method = pd.DataFrame([dict(settingid=settingid, **json.load(open("../" + setting["method_location"]))["params"]) for settingid, setting in enumerate(settings)])
    
    modeval_functional.load(settings_name)

    modeval_knownmodules = ModevalKnownmodules(settings)
    modeval_knownmodules.load(settings_name)
    modeval_coverage = ModevalCoverage(settings)
    modeval_coverage.load(settings_name)

    modeval_coverage.scores["refscore"] = modeval_coverage.scores["aucodds_permuted"]
    modeval_knownmodules.scores["refscore"] = modeval_knownmodules.scores["F1rprr_permuted"]
    modeval_knownmodules.scores["moduledef"] = [modulesname if modulesname in ["minimal", "strict"] else "interconnected" for modulesname in modeval_knownmodules.scores["knownmodules_name"]]

    refscores = pd.concat([modeval_knownmodules.scores, modeval_coverage.scores], ignore_index=True)[["settingid", "refscore", "goldstandard", "moduledef"]]
    refscores = refscores.merge(settings_dataset, on="settingid")
    
    proxyscores = modeval_functional.scores.rename(columns={"score":"proxyscore", "scorename":"proxyname"})
    
    combined = refscores.merge(proxyscores, on="settingid")
    
    scores[methodname] = combined

In [23]:
optimalproxy_funcs = {"asw":np.nanargmax, "ch":np.nanargmax, "dbindex":np.nanargmin, "dbstarindex":np.nanargmin, "bhi#go":np.nanargmax, "bhi#kegg":np.nanargmax, "bhi#reactome":np.nanargmax, 
                      "faucodds#go":np.nanargmax, "faucodds#kegg":np.nanargmax, "faucodds#reactome":np.nanargmax}

In [18]:
subscores["proxyscore"].idxmax()

NameError: name 'subscores' is not defined

In [25]:
finalscores = []

for method, methodscores in scores.items():
    methodscores["proxyscore"][np.isinf(methodscores["proxyscore"])] = np.nan
    for (dataset, goldstandard, proxyname), subscores in methodscores.groupby(("datasetname", "goldstandard", "proxyname")):
        if proxyname not in optimalproxy_funcs.keys():
            continue
        if proxyname.endswith("reactome"):
            continue

        optimalproxy_func = optimalproxy_funcs[proxyname]
        if np.all(np.isnan(subscores["proxyscore"])):
            print("random?", method)
            print(dataset, refgrouper,proxygrouper)
            optimalproxy = np.random.choice(subscores.index)
        else:
            optimalproxy = subscores.index[optimalproxy_func(subscores["proxyscore"])]
            
        row = subscores.ix[optimalproxy].copy()
        row["optimalrefscore"] = subscores["refscore"].max()
        row["method"] = method
        finalscores.append(row)
finalscores = pd.DataFrame(finalscores).rename(columns={"organism":"organismoi", "goldstandard":"goldstandardoi", "refscore":"score"})

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [26]:
from collections import Counter

In [29]:
proxyscores_ = add_weights(finalscores)
proxyscores_.groupby(("proxyname", "method")).apply(lambda x: np.average(x.score, weights=x.weight))

proxyname      method
bhi#go         agglom    2.673693
               dummy     0.352089
bhi#kegg       agglom    2.097965
               dummy     0.352089
faucodds#go    dummy     0.452503
faucodds#kegg  dummy     0.549578
dtype: float64

In [386]:
proxyscores_.groupby("method").apply(lambda x: np.average(x.optimalrefscore, weights=x.weight))

method
agglom    7.225501
dummy     0.589433
dtype: float64

In [28]:
def add_weights(scores):
    weights = []
    scores["moduledef"] = scores["moduledef"].fillna("")
    for organismoi, subscores in scores.groupby("organismoi"):
        moduledef_weights = 1/subscores.groupby("moduledef")["score"].count()
        for moduledef, weight in moduledef_weights.items():
            weights.append({
                    "organism":organismoi,
                    "moduledef":moduledef,
                    "weight":weight / len(moduledef_weights)
                })
    weights = pd.DataFrame(weights).set_index(["organism", "moduledef"])["weight"]
    
    scores["weight"] = weights.ix[pd.Index(scores[["organismoi", "moduledef"]])].tolist()
    
    return scores

In [383]:
scatter(scores["refscore"], scores["proxyscore"])

KeyError: 'refscore'